In [7]:
import numpy as np
import time
import tqdm
import json
import logging
import argparse

from pathlib import Path

from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, CSVLogger

import numpy as np
import math


In [2]:
ls './dataset/merger/test/'

587722982838436141.jpeg  587733080274108508.jpeg  587739827677954139.jpeg
587722982838436142.jpeg  587733080280203402.jpeg  587739827677954141.jpeg
587722982842630917.jpeg  587733080280203406.jpeg  587739828741144682.jpeg
587722982842630918.jpeg  587733081346801752.jpeg  587739828741144683.jpeg
587722982843613716.jpeg  587733081346801754.jpeg  587739829276508335.jpeg
587722982843613717.jpeg  587733081880002670.jpeg  587739829276508336.jpeg
587722983890944072.jpeg  587733081880002671.jpeg  587741386738303371.jpeg
587722983890944073.jpeg  587733196769984638.jpeg  587741386738303372.jpeg
587722983899791505.jpeg  587733196769984639.jpeg  587741387272683989.jpeg
587722983899791506.jpeg  587733398639870073.jpeg  587741387272683991.jpeg
587722983915716721.jpeg  587733398639870074.jpeg  587741420562219333.jpeg
587722983915716722.jpeg  587733398648455384.jpeg  587741420562219334.jpeg
587722984438104266.jpeg  587733398648455385.jpeg  587741422171259290.jpeg
587722984438104267.jpeg  5877333991794

In [3]:
import tensorflow as tf; print(tf.__version__)

1.4.0


In [4]:
import keras
print(keras.__version__)


2.2.4


In [8]:
def one_hot_array(index, length):

    one_hot = np.zeros(length)
    one_hot[index] = 1

    return one_hot

def make_stratified_generator(class_paths, target_size=(299, 299), individual_batchsize=8, zoom_factor=2., translation_range=0.05):

    zoom_base = math.sqrt(2) * ((1 - translation_range) ** -1)

    generators = []
    for class_path in class_paths:

        image_data_generator = image.ImageDataGenerator(rotation_range=360,
                                                        zoom_range=(zoom_base, zoom_base * zoom_factor),
                                                        width_shift_range=translation_range,
                                                        height_shift_range=translation_range,
                                                        vertical_flip=True)

        generator = image_data_generator.flow_from_directory(class_path,
                                                             batch_size=individual_batchsize,
                                                             target_size=target_size)
        generators += [generator]

    while True:

        images = []
        for generator in generators:
            images += [next(generator)][0]

        cumulative_batch = np.vstack(images)

        cumulative_labels = np.vstack(
                [np.tile(one_hot_array(i, len(class_paths)), (len(imgs), 1)) for i, imgs in enumerate(images)]
                )

        yield cumulative_batch, cumulative_lables

In [9]:
def write_status(filename, epoch_nr, training_error, validation_error, training_loss, validation_loss):

    info = {'epoch_nr': epoch_nr,
            'timestamp': time.time(),
            'training_error': training_error,
            'validation_error': validation_error,
            'training_loss': training_loss,
            'validation_loss': validation_loss}

    with open(filename, 'w') as fp:
        json.dump(info, fp)


def make_generators(path_images, positive_class='merger', negative_class='noninteracting'):

    training_gen = make_stratified_generator(Path(path_images) / positive_class / 'training',
                                             Path(path_images) / negative_class / 'training')

    validation_gen = make_stratified_generator(Path(path_images) / positive_class / 'validation',
                                               Path(path_images) / negative_class / 'validation')

    return training_gen, validation_gen


def training_loop(model, nr_epochs, path_images, training_type, steps_per_epoch = (6000 * 2/4), validation_steps = (6000 * 1/4)):

    training_gen, validation_gen=make_generators(path_images)

    for epoch in tqdm.tqdm(range(nr_epochs), total = nr_epochs):

        logging.info("starting {training_type} epoch #{epoch}".format(training_type, epoch))
        history=model.fit_generator(training_gen, steps_per_epoch = steps_per_epoch, epochs = 1,
                                    validation_data = validation_gen, validation_steps = validation_steps)
        logging.info("finished {training_type} epoch #{epoch}".format(training_type, epoch))

        model.save(Path(path_checkpoints) / '{training_type}_{epoch}.checkpoint'.format(training_type, epoch))

        training_error, validation_error = (1. - history.history['acc'][0]), (1. - history.history['val_acc'][0])
        training_loss, validation_loss = history.history['loss'][0], history.history['val_loss'][0]

        write_status(Path(path_statuses) / '{training_type}_{epoch}.status'.format(training_type, epoch),
                     epoch,
                     training_error,
                     validation_error,
                     training_loss,
                     validation_loss)

        return model


def preparation_training(path_images, path_checkpoints, path_statuses, nr_epochs = 40):

    if mode == 'transferlearning':
        xception=Xception(weights = 'imagenet', include_top = False)
    else:
        xception=Xception(weights = None, include_top = False)

    output=GlobalAveragePooling2D()(xception.output)
    output=Dense(1024, activation = 'relu')(output)
    output=Dense(2, activation = 'softmax')(output)
    model=Model(input = xception.input, output = output)

    for layer in xception.layers:
        layer.trainable=False  # freeze all layers except the new FC layers at the end

    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                  metrics=['accuracy'])

    model = training_loop(model, nr_epochs, path_images, training_type='preparation')

    model.save(Path(path_checkpoints) / 'main_0.checkpoint')


def main_training(path_images, path_checkpoints, path_statuses, current_epoch, nr_epochs=100000):

    current_checkpoint_filename = Path(path_checkpoints) / 'main_{current_epoch}.checkpoint'.format(current_epoch)
    model = load_model(current_checkpoint_filename)

    for layer in model.layers:
       layer.trainable = True # unfreeze all layers

    model.compile(optimizer=SGD(lr=0.5 * 3e-5, momentum=0.9), loss='categorical_crossentropy',
                  metrics=['accuracy'])

    training_loop(model, nr_epochs, path_images, training_type='main')


if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument('--path_images', required=True)
    parser.add_argument('--path_checkpoints', required=True)
    parser.add_argument('--path_statuses', required=True)
    parser.add_argument('--mode', required=True)
    args = parser.parse_args()

    path_images, path_checkpoints, path_statuses, mode = (args.path_images,
                                                          args.path_checkpoints,
                                                          args.path_statuses,
                                                          args.mode)

    logging.basicConfig(filename='LOG.log', level=logging.DEBUG)


    logging.info(("started with parameters",
                 "path_images: '{path_images}''".format(path_images),
                 "path_checkpoints: '{path_checkpoints}''".format(path_checkpoints),
                 "mode: '{mode}''".format(mode)))


    logging.info("starting new training")
    preparation_training(path_images, path_checkpoints, path_statuses, mode)

    logging.info("starting main training")
    main_training(path_images, path_checkpoints, path_statuses, 0)

usage: ipykernel_launcher.py [-h] --path_images PATH_IMAGES --path_checkpoints
                             PATH_CHECKPOINTS --path_statuses PATH_STATUSES
                             --mode MODE
ipykernel_launcher.py: error: the following arguments are required: --path_images, --path_checkpoints, --path_statuses, --mode


SystemExit: 2

/home/pap408/pyenv/py3.5.3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [27]:
import tensorflow as tf

# Make a queue of file names including all the JPEG images files in the relative
# image directory.
filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once('./dataset/merger/test/'))

# Read an entire image file which is required since they're JPEGs, if the images
# are too large they could be split in advance to smaller files or use the Fixed
# reader to split up the file.
image_reader = tf.WholeFileReader()

# Read a whole file from the queue, the first returned value in the tuple is the
# filename which we are ignoring.
_, image_file = image_reader.read(filename_queue)

# Decode the image as a JPEG file, this will turn it into a Tensor which we can
# then use in training.
image = tf.image.decode_jpeg(image_file)

# Start a new session to show example output.
with tf.Session() as sess:
    # Required to get the filename matching to run.
#     tf.initialize_all_variables().run()
    tf.global_variables_initializer().run()

    # Coordinate the loading of image files.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    # Get an image tensor and print its value.
    image_tensor = sess.run([image])
    print(image_tensor)

    # Finish off the filename queue coordinator.
    coord.request_stop()
    coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.FailedPreconditionError'>, Attempting to use uninitialized value matching_filenames
	 [[Node: matching_filenames/read = Identity[T=DT_STRING, _class=["loc:@matching_filenames"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](matching_filenames)]]


OutOfRangeError: FIFOQueue '_3_input_producer_1' is closed and has insufficient elements (requested 1, current size 0)
	 [[Node: ReaderReadV2_1 = ReaderReadV2[_device="/job:localhost/replica:0/task:0/device:CPU:0"](WholeFileReaderV2_1, input_producer_1)]]

Caused by op 'ReaderReadV2_1', defined at:
  File "/share/apps/python3/3.5.3/intel/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/share/apps/python3/3.5.3/intel/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/pap408/pyenv/py3.5.3/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/pap408/pyenv/py3.5.3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/pap408/pyenv/py3.5.3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/pap408/pyenv/py3.5.3/lib/python3.5/